In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = (4, 4)
plt.rcParams['figure.dpi'] = 150
plt.rcParams['lines.linewidth'] = 3
sns.set()

## NBA

In [ ]:
import requests
import os

def fetch():
    path = 'nba.csv'
    if not os.path.exists(path):
        url = 'https://stats.nba.com/stats/leaguegamelog/'
        params = (
            ('Counter', '0'),
            ('DateFrom', ''),
            ('DateTo', ''),
            ('Direction', 'ASC'),
            ('LeagueID', '00'),
            ('PlayerOrTeam', 'T'),
            ('Season', '2017-18'),
            ('SeasonType', 'Regular Season'),
            ('Sorter', 'DATE'),
        )
        headers = {
            'User-Agent': 'PostmanRuntime/7.4.0'
        }
        response = requests.get(url, params=params, headers=headers)
        data = response.json()['resultSets'][0]
        df = pd.DataFrame(data=data['rowSet'], columns=data['headers'])
        df.to_csv(path, index=False)
        return df
    else:
        return pd.read_csv(path)
    
df = fetch()

## Logistic Regression

One way to model the relationship between vector-valued $X$ and the conditional distribution of a binary categorical $Y \in {0, 1}$ is to assume that the log odds ratio is linear in $X$:

$$\log \left( \frac{P(Y=1|X)}{P(Y=0|X)} \right) = X^T \beta$$

Abbreviate $P(Y=1|X)$ as $p$ and $X^T \beta$ as $t$, then:

\begin{align*}
\log \left( \frac{P(Y=1|X)}{P(Y=0|X)} \right) &= X^T \beta && \text{} \\[10pt]
\log \left( \frac{p}{1-p} \right) &= t && \text{; Abbreviate and complement rule} \\[10pt]
\frac{p}{1-p} &= \exp(t) && \text{; exponentiate both sides} \\[10pt]
p &= \exp(t) - p \exp(t) && \text{; multiply by $1-p$} \\[10pt]
p (1 + \exp(t)) &= \exp(t) && \text{; add $p \exp(t)$ and factor out $p$} \\[10pt]
p &= \frac{\exp(t)}{1 + \exp(t)} && \text{; divide by $1 + \exp(t)$} \\[10pt]
p &= \frac{1}{1 + \exp(-t)} && \text{; multiply by $\frac{\exp(-t)}{\exp(-t)}$} \\[10pt]
P(Y=1|X) &= \frac{1}{1 + \exp(-X^T \beta)} && \text{; Unabbreviate}
\end{align*}

This transformation is called the logistic function, traditionally denoted $\sigma$ (sigma).

$$\sigma(t) = \frac{1}{1 + \exp(-t)}$$

In [ ]:
def sigma(t):
    return 1 / (1 + np.exp(-t))

In [ ]:
def flatten(li): 
    return [item for sub in li for item in sub]

bs = [-2, -1, -0.5, 2, 1, 0.5]
xs = np.linspace(-10, 10, 100)

fig, axes = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(10, 6))
for ax, b in zip(flatten(axes), bs):
    ys = sigma(xs * b)
    ax.plot(xs, ys)
    ax.set_title(r'$ b = $' + str(b))

# add a big axes, hide frame
fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axes
plt.tick_params(labelcolor='none', top=False, bottom=False,
                left=False, right=False)
plt.grid(False)
plt.xlabel('$x$')
plt.ylabel(r'$ \frac{1}{1+\exp(-b \cdot x)} $')
plt.tight_layout()

## Log Loss (a.k.a., Cross-Entropy Loss)

In [ ]:
toy = pd.DataFrame({
    'X': [-5, -4, -3, -2, -1, 1, 2, 3, 4, 5],
    'Y': [ 0,  0,  0,  0,  1, 0, 1, 1, 1, 1]
})
plt.scatter(toy['X'], toy['Y']);

In [ ]:
def empirical_risk(b, loss):
    y_hats = sigma(toy['X'] * b)
    losses = [loss(y, y_hat) for y, y_hat in zip(toy['Y'], y_hats)]
    return np.average(losses)

def squared_loss(y, y_hat):
    return (y - y_hat) ** 2

bs = np.linspace(-4, 4, 50)

plt.plot(bs, [empirical_risk(b, squared_loss) for b in bs]);
plt.xlabel('b')
plt.ylabel('empirical risk')

In [ ]:
p_hat = np.arange(0.001, 0.999, 0.01)
loss = -np.log(p_hat)
plt.plot(p_hat, loss, color='k')
plt.xlabel('$\hat{p}$: Predicted Chance of Correct Class')
plt.ylabel('$-\log(\hat{p})$')
plt.title('Loss for One Individual');

In [ ]:
# Predicted chance of class 1.
x = np.arange(-5, 5, 0.01)
px = sigma(x)
plt.plot(x, px, color = 'darkblue')

def add_points(xs, y, color):
    plt.scatter(xs, [y] * len(xs), color=color, s=40)
    for x in xs:
        plt.plot([x, x], [1-y, sigma(x)], color=color)

add_points(np.array([-5, -4, -3, -2, 1]), 0, 'gold')
add_points(np.array([-1, 2, 3, 4, 5]), 1, 'darkblue')
        
plt.xlabel('x')
plt.title('Predicted Chance of Correct Class');

Log loss (a.k.a., cross-entropy loss)
$$
-y_i\log(\sigma(x_i^T \beta)) - (1-y_i)\log(1-\sigma(x_i^T \beta))
$$

In [ ]:
def log_loss(y, y_hat):
    return -y * np.log(y_hat) - (1-y) * np.log(1-y_hat)

plt.plot(bs, [empirical_risk(b, log_loss) for b in bs]);
plt.xlabel('b')
plt.ylabel('empirical risk')

## Applying Logistic Regression

In [ ]:
games.iloc[0]

In [ ]:
sns.jointplot('FG_PCT_DIFF', 'WON', 
              kind='reg', y_jitter=0.1, 
              logistic=True, ci=None,
              data=games);

In [ ]:
b = minimize(risk, 1)['x'][0]
win_rates.plot();
plt.plot(fg_pct_diffs, sigma(fg_pct_diffs * b));